In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=8, app_name='pdu_syp2', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:25


In [3]:
june_gcoin = pre_load_data_mart(start_date="2021-06-02", end_date="2021-06-29", device="pc", table_name="gcoin_use")
aug_gcoin = load_data_mart(start_date="2021-08-11", end_date="2021-09-07", device="pc", table_name="gcoin_use")
oct_gcoin = load_data_mart(start_date="2021-08-11", end_date="2021-09-07", device="pc", table_name="gcoin_use")

In [5]:
import pickle
with open("./data/june_wsus_user.pickle", "rb") as f:
    june_wsus_user = pickle.load(f)
with open("./data/aug_wsus_user.pickle", "rb") as f:
    aug_wsus_user = pickle.load(f)
with open("./data/oct_wsus_user.pickle", "rb") as f:
    oct_wsus_user = pickle.load(f)

In [13]:
june_gcoin_df = june_gcoin.where(col("account_id").isin(june_wsus_user)).select("date", "account_id", "product_id", "product_name", "paid_use").toPandas()
aug_gcoin_df = aug_gcoin.where(col("account_id").isin(aug_wsus_user)).select("date", "account_id", "product_id", "product_name", "paid_use").toPandas()
oct_gcoin_df = oct_gcoin.where(col("account_id").isin(oct_wsus_user)).select("date", "account_id", "product_id", "product_name", "paid_use").toPandas()

In [14]:
june_gcoin_df.to_csv("./data/june_gcoin_df.csv", index=False)
aug_gcoin_df.to_csv("./data/aug_gcoin_df.csv", index=False)
oct_gcoin_df.to_csv("./data/oct_gcoin_df.csv", index=False)

In [22]:
items = mysql.read_table(spark, "metainfo", "item_meta_sales").groupBy("product_id", "product_name", "part").agg(count("*")).toPandas()

In [15]:
june_item_equipped_df = pd.read_csv("./data/june_item_equipped_df.csv")
aug_item_equipped_df = pd.read_csv("./data/aug_item_equipped_df.csv")
oct_item_equipped_df = pd.read_csv("./data/oct_item_equipped_df.csv")

In [16]:
june_wsus_equip_user = june_item_equipped_df[june_item_equipped_df.ItemId == "itemdesc.12012001"].AccountId.unique().tolist()
aug_wsus_equip_user = aug_item_equipped_df[aug_item_equipped_df.ItemId == "itemdesc.12012002"].AccountId.unique().tolist()
oct_wsus_equip_user = oct_item_equipped_df[oct_item_equipped_df.ItemId == "itemdesc.12012003"].AccountId.unique().tolist()

In [23]:
june_item_equipped_df = pd.merge(june_item_equipped_df, items, left_on="ItemId", right_on="product_id")
aug_item_equipped_df = pd.merge(aug_item_equipped_df, items, left_on="ItemId", right_on="product_id")
oct_item_equipped_df = pd.merge(oct_item_equipped_df, items, left_on="ItemId", right_on="product_id")

#### 성장형 무기를 소유하고 있지만 성장형 무기를 사용하지 않고 해당 무기의 다른 스킨을 사용한 유저수

In [28]:
june_item_equipped_df[(~june_item_equipped_df.AccountId.isin(june_wsus_equip_user)) & (june_item_equipped_df.part == "m762")].AccountId.nunique()

107

In [45]:
aug_item_equipped_df[(~aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user)) & (aug_item_equipped_df.part == "fnfal")].AccountId.nunique()

59

In [46]:
oct_item_equipped_df[(~oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user)) & (oct_item_equipped_df.part == "m416")].AccountId.nunique()

114

In [30]:
# 성장형 무기를 소유하고 있지만 성장형 무기를 사용하지 않고 해당 무기의 다른 스킨을 사용한 유저
june_tmp_user_list = june_item_equipped_df[(~june_item_equipped_df.AccountId.isin(june_wsus_equip_user)) & (june_item_equipped_df.part == "m762")].AccountId.unique().tolist()
aug_tmp_user_list = aug_item_equipped_df[(~aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user)) & (aug_item_equipped_df.part == "fnfal")].AccountId.unique().tolist()
oct_tmp_user_list = oct_item_equipped_df[(~oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user)) & (oct_item_equipped_df.part == "m416")].AccountId.unique().tolist()

In [54]:
114/69630*100

0.16372253339077983

#### 성장형 무기 소유하고 있지만 사용하지 않은 사용자들의 무기 스킨 사용

In [49]:
june_item_equipped_df[june_item_equipped_df.AccountId.isin(june_tmp_user_list)].groupby("product_name")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False).head(10)

,product_name,AccountId
23,Gold Plate - Beryl M762,72
10,CNH's SLR,28
75,Silver Plate - Beryl M762,27
76,Silver Plate - M416,23
24,Gold Plate - M416,21
73,Seahorse Cult - Beryl M762,14
0,17SHOU's Beryl M762,13
58,PCS4 Gold Mettle - Pan,12
26,Happy Pew Year - Beryl M762,10
64,PGI.S Hydrodip - M416,8


In [50]:
aug_item_equipped_df[aug_item_equipped_df.AccountId.isin(aug_tmp_user_list)].groupby("product_name")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False).head(10)

,product_name,AccountId
13,CNH's SLR,31
89,[PROGRESSIVE] Gear Head - Beryl M762,29
36,LongSkr's AKM,13
79,Silver Plate - M416,13
46,MinORu's Mini14,12
44,Manticore - SLR,11
84,Swagger's SLR,9
82,Son Heung-min - Mini14,8
90,chocoTaco's Beryl M762,8
57,PCS3 Blue Bullion - SLR,8


In [51]:
oct_item_equipped_df[oct_item_equipped_df.AccountId.isin(oct_tmp_user_list)].groupby("product_name")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False).head(10)

,product_name,AccountId
11,Bunny Aeronautics - M416,37
111,[PROGRESSIVE] Gear Head - Beryl M762,35
94,Silver Plate - M416,33
110,[PROGRESSIVE] Buzzkill - SLR,31
57,NeoticaCH's M416,25
27,Gold Plate - M416,20
105,White Rabbit - Pan,19
76,PGI.S Hydrodip - M416,15
93,Silver Plate - Beryl M762,14
12,Bunny Aeronautics - Mini14,14


### 성장형 무기를 소유하고 있지만 해당 무기스킨을 아예 안쓰고 다른 무기 쓰는 유저


In [59]:
june_item_equipped_df[(~june_item_equipped_df.AccountId.isin(june_wsus_equip_user)) & (~june_item_equipped_df.AccountId.isin(june_tmp_user_list))]["AccountId"].nunique()

228

In [60]:
aug_item_equipped_df[(~aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user)) & (~aug_item_equipped_df.AccountId.isin(aug_tmp_user_list))]["AccountId"].nunique()

155

In [61]:
oct_item_equipped_df[(~oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user)) & (~oct_item_equipped_df.AccountId.isin(oct_tmp_user_list))]["AccountId"].nunique()

278

In [37]:
june_item_equipped_df[(~june_item_equipped_df.AccountId.isin(june_wsus_equip_user)) & (~june_item_equipped_df.AccountId.isin(june_tmp_user_list))].groupby("part")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False)

,part,AccountId
4,fnfal,97
11,m416,66
19,pan,42
0,akm,36
14,mini14,27
9,m24,25
7,kar98,19
25,sks,17
26,ump9,12
20,qbz95,8


In [41]:
june_item_equipped_df[(~june_item_equipped_df.AccountId.isin(june_wsus_equip_user)) & (~june_item_equipped_df.AccountId.isin(june_tmp_user_list))].groupby("product_name")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False).head(10)

,product_name,AccountId
12,CNH's SLR,74
53,PCS4 Gold Mettle - Pan,24
23,Gold Plate - M416,22
32,MOZZ's M24,19
39,MinORu's Mini14,16
43,PCS1 - M416,13
74,Silver Plate - M416,12
54,PCS4 Gold Mettle - UMP45,11
55,PGC 2019 - M416,10
50,PCS2 Gilded Triumph - Mini14,10


In [38]:
aug_item_equipped_df[(~aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user)) & (~aug_item_equipped_df.AccountId.isin(aug_tmp_user_list))].groupby("part")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False)

,part,AccountId
11,m762,91
10,m416,56
0,akm,41
6,kar98,27
8,m24,16
19,pan,16
14,mini14,16
22,sks,10
15,mk47,6
16,mosin_nagant,5


In [42]:
aug_item_equipped_df[(~aug_item_equipped_df.AccountId.isin(aug_wsus_equip_user)) & (~aug_item_equipped_df.AccountId.isin(aug_tmp_user_list))].groupby("product_name")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False).head(10)

,product_name,AccountId
78,[PROGRESSIVE] Gear Head - Beryl M762,74
68,Silver Plate - M416,21
69,Son Heung-min - M416,16
17,Gold Plate - Beryl M762,15
26,LongSkr's AKM,12
67,Silver Plate - Beryl M762,12
18,Gold Plate - M416,10
3,Ashek's AKM,9
1,Animal House - AKM,8
77,XingHun s AKM,8


In [39]:
oct_item_equipped_df[(~oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user)) & (~oct_item_equipped_df.AccountId.isin(oct_tmp_user_list))].groupby("part")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False)

,part,AccountId
7,fnfal,110
14,m762,107
23,pan,75
0,akm,41
12,m24,39
13,m249,26
10,kar98,24
4,crossbow,22
17,mini14,20
26,s686,15


In [44]:
oct_item_equipped_df[(~oct_item_equipped_df.AccountId.isin(oct_wsus_equip_user)) & (~oct_item_equipped_df.AccountId.isin(oct_tmp_user_list))].groupby("product_name")["AccountId"].nunique().reset_index().sort_values("AccountId", ascending=False).head(10)

,product_name,AccountId
98,[PROGRESSIVE] Gear Head - Beryl M762,74
97,[PROGRESSIVE] Buzzkill - SLR,70
93,White Rabbit - Pan,40
13,CNH's SLR,22
53,Night Slayer - Crossbow,22
50,Molten Mayhem - M249,21
27,Gold Plate - Beryl M762,20
42,MOZZ's M24,16
41,Lycan Hunter - S686,15
84,Silver Plate - Beryl M762,14
